In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import LSTM, Dense, SimpleRNN
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df=pd.read_excel('/content/drive/MyDrive/Agritech/Agmarknet_Price_And_Arrival_Report_MP.xlsx', header=1)
df.head()

,State Name,District Name,Market Name,Variety,Group,Arrivals (Tonnes),Min Price (Rs./Quintal),Max Price (Rs./Quintal),Modal Price (Rs./Quintal),Reported Date
0,Madhya Pradesh,Ashoknagar,Ashoknagar,Sharbati,Cereals,71.25,2880,4665,2230,2023-12-30
1,Madhya Pradesh,Ashoknagar,Ashoknagar,Lokwan,Cereals,210.83,2891,2891,2288,2023-12-29
2,Madhya Pradesh,Ashoknagar,Ashoknagar,Sharbati,Cereals,NaN,3085,4701,2288,2023-12-29
3,Madhya Pradesh,Ashoknagar,Ashoknagar,Other,Cereals,NaN,3150,3150,2288,2023-12-29
4,Madhya Pradesh,Ashoknagar,Ashoknagar,Sharbati,Cereals,165.80,2776,4666,2500,2023-12-28


In [ ]:
df['Reported Date']=pd.to_datetime(df['Reported Date'])
df['Year']=df['Reported Date'].dt.year
df.head()

,State Name,District Name,Market Name,Variety,Group,Arrivals (Tonnes),Min Price (Rs./Quintal),Max Price (Rs./Quintal),Modal Price (Rs./Quintal),Reported Date,Year
0,Madhya Pradesh,Ashoknagar,Ashoknagar,Sharbati,Cereals,71.25,2880,4665,2230,2023-12-30,2023
1,Madhya Pradesh,Ashoknagar,Ashoknagar,Lokwan,Cereals,210.83,2891,2891,2288,2023-12-29,2023
2,Madhya Pradesh,Ashoknagar,Ashoknagar,Sharbati,Cereals,NaN,3085,4701,2288,2023-12-29,2023
3,Madhya Pradesh,Ashoknagar,Ashoknagar,Other,Cereals,NaN,3150,3150,2288,2023-12-29,2023
4,Madhya Pradesh,Ashoknagar,Ashoknagar,Sharbati,Cereals,165.80,2776,4666,2500,2023-12-28,2023


In [ ]:
df_train=df[df['Year']!=2023]
df_test=df[df['Year']==2023]
df_test['Year'].unique()

array([2023], dtype=int32)

In [ ]:
from datetime import time
class Crop_Price_Prediction:
  def __init__(self, df_train, df_test):
    self.df_train=df_train
    self.df_test=df_test
    self.model=load_model('/content/drive/MyDrive/Agritech/crop_model.h5')
    # self.train=self.df[self.df['']]

  def prepare_train_data(self, n_features):

    timeseries=self.df_train['Modal Price (Rs./Quintal)'].values.reshape(-1, 1)
    self.scaler = MinMaxScaler()
    self.timeseries = self.scaler.fit_transform(timeseries)
    X,y=[], []
    for i in range(len(self.timeseries)):
      end_ix = i + n_features

      if end_ix > len(self.timeseries)-1:
        break

      seq_x, seq_y= self.timeseries[i:end_ix], self.timeseries[end_ix]
      X.append(seq_x)
      y.append(seq_y)

    return np.array(X), np.array(y)


  def head(self):
    return self.df.head()


  def lstm_model(self, n_steps=4, n_features=1):
    self.model = Sequential()
    self.model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
    self.model.add(LSTM(50, activation = 'relu'))
    self.model.add(Dense(1))
    self.model.compile(optimizer='adam', loss='mse')

  def lstm_training(self, X_train, y_train):
    self.model.fit(X_train, y_train, epochs=10)
    self.model.save('/content/drive/MyDrive/Agritech/crop_model.h5')

  def lstm_predict(self, X_input, n_features=4):

        # self.scaler = MinMaxScaler()
        new_data_scaled = self.scaler.transform(X_input)

        # Reshape the new data into sequences
        # X_new = []
        # for i in range(len(new_data_scaled)):
        #     end_ix = i + n_features
        #     if end_ix > len(new_data_scaled) - 1:
        #         break
        #     seq_x = new_data_scaled[i:end_ix]
        #     X_new.append(seq_x)
        # X_new = np.array(X_new)

        # Make predictions
        predictions_scaled = self.model.predict(X_new)

        # Inverse transform the predictions to get actual values
        predictions = self.scaler.inverse_transform(predictions_scaled)

        return predictions

cp=Crop_Price_Prediction(df_train, df_test)

In [ ]:
df_train['Year'].values

array([2022, 2022, 2022, ..., 2016, 2016, 2016], dtype=int32)

In [ ]:
X_train, y_train = cp.prepare_train_data(n_features=4)

In [ ]:
X_train.shape, y_train.shape

((8012, 4, 1), (8012, 1))

In [ ]:
X_train=X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_train.shape

(8012, 4, 1)

In [ ]:
cp.lstm_model()
cp.lstm_training(X_train, y_train)

Epoch 1/10
251/251 [==============================] - 4s 6ms/step - loss: 0.0133
Epoch 2/10
251/251 [==============================] - 2s 7ms/step - loss: 0.0076
Epoch 3/10
251/251 [==============================] - 4s 14ms/step - loss: 0.0075
Epoch 4/10
251/251 [==============================] - 2s 6ms/step - loss: 0.0074
Epoch 5/10
251/251 [==============================] - 2s 6ms/step - loss: 0.0074
Epoch 6/10
251/251 [==============================] - 2s 6ms/step - loss: 0.0072
Epoch 7/10
251/251 [==============================] - 2s 6ms/step - loss: 0.0067
Epoch 8/10
251/251 [==============================] - 2s 6ms/step - loss: 0.0063
Epoch 9/10
251/251 [==============================] - 2s 6ms/step - loss: 0.0059
Epoch 10/10
251/251 [==============================] - 2s 9ms/step - loss: 0.0054


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# X_test, y_test = cp.prepare_test_data(30)

In [ ]:


# X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [ ]:
# X_test.shape


In [ ]:
a=cp.lstm_predict(X_input=df_test['Modal Price (Rs./Quintal)'].values.reshape(-1, 1))

24/24 [==============================] - 1s 6ms/step


In [ ]:
a.shape

(737, 1)

In [ ]:
df_test.shape

(741, 11)

In [ ]:
df.head()

,State Name,District Name,Market Name,Variety,Group,Arrivals (Tonnes),Min Price (Rs./Quintal),Max Price (Rs./Quintal),Modal Price (Rs./Quintal),Reported Date,Year
0,Madhya Pradesh,Ashoknagar,Ashoknagar,Sharbati,Cereals,71.25,2880,4665,2230,2023-12-30,2023
1,Madhya Pradesh,Ashoknagar,Ashoknagar,Lokwan,Cereals,210.83,2891,2891,2288,2023-12-29,2023
2,Madhya Pradesh,Ashoknagar,Ashoknagar,Sharbati,Cereals,NaN,3085,4701,2288,2023-12-29,2023
3,Madhya Pradesh,Ashoknagar,Ashoknagar,Other,Cereals,NaN,3150,3150,2288,2023-12-29,2023
4,Madhya Pradesh,Ashoknagar,Ashoknagar,Sharbati,Cereals,165.80,2776,4666,2500,2023-12-28,2023
